In [1]:
from typing import Optional, List, Dict, Any, Union
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

In [2]:
# First, define a proper state class with all needed fields
class AgentState(BaseModel):
    """State for the travel assistant workflow."""
    query: str = ""
    agent_executor: Optional[str] = None
    response: Optional[str] = None
    error: Optional[str] = None
    context: Dict[str, Any] = Field(default_factory=dict)
    messages: List[BaseMessage] = Field(default_factory=list)

# Define the agent functions that properly update the state
def router_agent(state: AgentState) -> AgentState:
    """Routes the query to the appropriate agent."""
    query = state.query.lower()
    
    if any(word in query for word in ["itinerary", "plan", "trip", "day"]):
        return {"agent_executor": "itinerary_agent"}
    elif any(word in query for word in ["flight", "fly", "airline", "airport"]):
        return {"agent_executor": "flight_agent"}
    elif any(word in query for word in ["hotel", "stay", "accommodation", "book", "reserve"]):
        return {"agent_executor": "accommodation_agent"}
    else:
        return {"agent_executor": "information_agent"}

def itinerary_agent(state: AgentState) -> AgentState:
    """Handles itinerary planning requests."""
    try:
        # Process the itinerary request
        response = f"Here's a suggested itinerary for {state.query}: [Itinerary details would go here]"
        return {"response": response}
    except Exception as e:
        return {"error": f"Error in itinerary planning: {str(e)}"}

def flight_agent(state: AgentState) -> AgentState:
    """Handles flight booking requests."""
    try:
        # Process the flight request
        response = f"Here are the flight options for {state.query}: [Flight details would go here]"
        return {"response": response}
    except Exception as e:
        return {"error": f"Error in flight search: {str(e)}"}

def accommodation_agent(state: AgentState) -> AgentState:
    """Handles accommodation booking requests."""
    try:
        # Process the accommodation request
        response = f"Here are the accommodation options for {state.query}: [Accommodation details would go here]"
        return {"response": response}
    except Exception as e:
        return {"error": f"Error in accommodation search: {str(e)}"}

def information_agent(state: AgentState) -> AgentState:
    """Handles general travel information requests."""
    try:
        # Process the information request
        response = f"Here's the information about {state.query}: [Travel information would go here]"
        return {"response": response}
    except Exception as e:
        return {"error": f"Error retrieving information: {str(e)}"}

def generate_final_response(state: AgentState) -> AgentState:
    """Generates the final response to the user."""
    # You could do additional formatting here
    return {"response": f"✅ {state.response}"}

def handle_error(state: AgentState) -> AgentState:
    """Handles errors that occurred during processing."""
    return {"response": f"I'm sorry, but I encountered an error: {state.error}"}

def create_travel_assistant_graph():
    """Creates the travel assistant graph using LangGraph."""
    # Initialize the workflow graph
    workflow = StateGraph(AgentState)
    
    # Add nodes for each agent and processing step
    workflow.add_node("router", router_agent)
    workflow.add_node("itinerary_agent", itinerary_agent)
    workflow.add_node("flight_agent", flight_agent)
    workflow.add_node("accommodation_agent", accommodation_agent)
    workflow.add_node("information_agent", information_agent)
    workflow.add_node("response_generator", generate_final_response)
    workflow.add_node("error_handler", handle_error)
    
    # Define conditional edge routing
    def router_edges(state):
        if state.agent_executor == "itinerary_agent":
            return "itinerary_agent"
        elif state.agent_executor == "flight_agent":
            return "flight_agent"
        elif state.agent_executor == "accommodation_agent":
            return "accommodation_agent"
        else:
            return "information_agent"
    
    def agent_edges(state):
        if state.error is not None:
            return "error_handler"
        else:
            return "response_generator"
    
    # Set entry point
    workflow.set_entry_point("router")
    
    # Connect router to agents
    workflow.add_conditional_edges("router", router_edges)
    
    # Connect agents to next nodes
    for agent in ["itinerary_agent", "flight_agent", "accommodation_agent", "information_agent"]:
        workflow.add_conditional_edges(agent, agent_edges)
    
    # Connect to end
    workflow.add_edge("response_generator", END)
    workflow.add_edge("error_handler", END)
    
    # Compile the graph
    travel_assistant = workflow.compile()
    return travel_assistant

# Create the graph
travel_assistant = create_travel_assistant_graph()



In [6]:
# Test function
def test_travel_assistant():
    test_queries = [
        "Plan a 5-day trip to Tokyo in March",
        "Find me flights from New York to London next week",
        "Book a hotel in Paris near the Eiffel Tower",
        "What's the weather like in Rome in July?"
    ]
    
    for query in test_queries:
        print(f"\nTesting: {query}")
        result = travel_assistant.invoke({"query": query})
        print(f"Routed to: {result['agent_executor']}")
        print(f"Response: {result['response']}")


In [4]:
result = travel_assistant.invoke({"query": "Plan a 5-day trip to Tokyo in March"})
print(result)

{'query': 'Plan a 5-day trip to Tokyo in March', 'agent_executor': 'itinerary_agent', 'response': "✅ Here's a suggested itinerary for Plan a 5-day trip to Tokyo in March: [Itinerary details would go here]"}


In [7]:
test_travel_assistant()


Testing: Plan a 5-day trip to Tokyo in March
Routed to: itinerary_agent
Response: ✅ Here's a suggested itinerary for Plan a 5-day trip to Tokyo in March: [Itinerary details would go here]

Testing: Find me flights from New York to London next week
Routed to: flight_agent
Response: ✅ Here are the flight options for Find me flights from New York to London next week: [Flight details would go here]

Testing: Book a hotel in Paris near the Eiffel Tower
Routed to: accommodation_agent
Response: ✅ Here are the accommodation options for Book a hotel in Paris near the Eiffel Tower: [Accommodation details would go here]

Testing: What's the weather like in Rome in July?
Routed to: information_agent
Response: ✅ Here's the information about What's the weather like in Rome in July?: [Travel information would go here]
